In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
import csv

# Load data
ordini_per_cliente = json.load(open('data/utils/ordini_per_client.json'))       
tot_per_cliente = json.load(open('data/utils/tot_per_client.json')) 
quantity = pd.read_csv('data/utils/quantity_per_client.csv')
print(f'prima {quantity.shape}')

job_machines = json.load(open('data/utils/articoli_macchine.json'))
job_list_temp = [job for job in job_machines] 

#Filtraggio degli articoli che non hanno dati presenti su "lista_articoli"
job_list = []
with open('data/valid/lista_articoli.csv', newline='') as csvfile:
    csv_data = csv.reader(csvfile, delimiter=',', quotechar='"')
    #row[0] è il codarticolo
    for idx, row in enumerate(csv_data):
        if job_list_temp.count(row[0]) > 0:
            job_list.append(row[0])

#Filtraggio degli ordini che NON presentano almeno una associazione macchina
o_p_c_filtrata = {}
for cliente in ordini_per_cliente:
    dict_ordini = ordini_per_cliente[cliente] 
    for ordine in dict_ordini:
        if job_list.count(ordine) > 0:
            if cliente not in o_p_c_filtrata:
                o_p_c_filtrata[cliente] = {}
            # Append the new value to the list associated with the key
            o_p_c_filtrata[cliente][ordine] = ordini_per_cliente[cliente][ordine]
        # else:
        #     print(f'per cliente: {cliente}, ordine: {ordine}, non trovato')

#FIltraggio degli 


t_p_c_filtrata = {}
for cliente in tot_per_cliente:
    if o_p_c_filtrata.get(cliente) is not None:
        t_p_c_filtrata[cliente] = tot_per_cliente[cliente]

quantity_filtrata = {'Cliente': [], 'cod articolo': [], 'mean': [], 'std': [], 'mean_differenza_giorni': []}
for index, row in quantity.iterrows():
    # print(f"Row {index}: {row.to_dict()}")
    if t_p_c_filtrata.get(row['Cliente']) is not None and job_list.count(row['cod articolo']) > 0:
        quantity_filtrata['Cliente'].append(row['Cliente'])
        quantity_filtrata['cod articolo'].append(row['cod articolo'])
        quantity_filtrata['mean'].append(row['mean'])
        quantity_filtrata['std'].append(row['std'])
        quantity_filtrata['mean_differenza_giorni'].append(row['mean_differenza_giorni'])

quantity = pd.DataFrame(quantity_filtrata)
print(f'dopo {quantity.shape}')

print(f'prima job_list_temp: {len(job_list_temp)}, dopo job_list: {len(job_list)}')
print(f'prima: {len(ordini_per_cliente)}, dopo {len(o_p_c_filtrata)}')
print(ordini_per_cliente['GT 2000'])
print(o_p_c_filtrata.get('GT 2000'))

print(f'prima: {len(tot_per_cliente)}, dopo {len(t_p_c_filtrata)}')

# Utils
clienti = list(o_p_c_filtrata.keys())               # Lista di clienti
tot = sum(t_p_c_filtrata.values())                     # Totale ordini per cliente nello storico
norm = {k: v/tot for k,v in t_p_c_filtrata.items()}    # Probabilità per cliente di comparire nello storico

prima (316, 5)
dopo (98, 5)
prima job_list_temp: 121, dopo job_list: 93
prima: 87, dopo 39
{'2218ZN': 1}
None
prima: 87, dopo 39


In [ ]:
### Generate synthetic data

## params
num_samples = 10       # Number of samples to generate

# Set to True to generate data starting from today (orders in entrata oggi)
today = False       

# Altrimenti imposta un range
start = '2024-11-15'
end = '2024-11-15'

new_data = []

# Loop to generate synthetic data with progress bar
for _ in tqdm(range(num_samples), desc="Generating data"):

    # Scegli cliente in modo uniforme
    i = np.round(np.random.uniform(0, len(clienti) - 1))
    cliente = clienti[int(i)]
    prob = norm[cliente]
    num = int(np.round(prob * num_samples))                     # Numero di ordini da generare per il cliente
    rows = quantity[quantity['Cliente'] == cliente].shape[0]
    if rows > 0:                                                # Se il cliente ha articoli nel dataset
        articoli = quantity[quantity['Cliente'] == cliente]     # Filtra articoli per cliente
        
        for _ in range(num + 1):
            art_index = int(np.round(np.random.uniform(0, rows - 1)))   # Scegli un articolo in modo uniforme
            art = articoli.iloc[art_index]

            articolo = art['cod articolo']              # Nome dell'articolo
            quantity_mean = art['mean']                 # Media della quantità ordinata                
            quantity_std = art['std']                   # Deviazione standard della quantità ordinata
            giorni = art['mean_differenza_giorni']      # media(consegna - ordine) | articolo, cliente

            # Ensure kg is always positive
            kg = np.round(np.random.normal(quantity_mean, quantity_std))
            if kg <= 0:                                     # Se la quantità è negativa, 
                kg = quantity_mean                              # prendi la media
            
            if today:
                data_inizio = pd.Timestamp.today().date()
            else:
                date_range = pd.date_range(start=start, end=end)
                data_inizio = np.random.choice(date_range)
                data_inizio = pd.Timestamp(data_inizio).date()

            # Calculate the end date
            data_fine = data_inizio + pd.Timedelta(days=giorni)
            data_fine = pd.Timestamp(data_fine).date()

            # Append the generated data to new_data list
            new_data.append({                               # Inserisci come dizionario in una lista di ordini
                'cliente': cliente,
                'cod_articolo': articolo,
                'quantity': kg,
                'data inserimento': data_inizio,
                'data consegna': data_fine
            })

columns = ['cliente', 'cod_articolo', 'quantity', 'data inserimento', 'data consegna']
res = pd.DataFrame(data=new_data, columns=columns)

res.to_csv('data/new_orders.csv', index=False)
res

Generating data:   0%|          | 0/10 [00:00<?, ?it/s]

Generating data: 100%|██████████| 10/10 [00:00<00:00, 315.85it/s]


,cliente,cod_articolo,quantity,data inserimento,data consegna
0,CALZIFICIO CREMONA,X20-2222DN2 mini,400.0,2024-11-15,2024-12-11
1,MARYGOLD,1724S+ZN 2cicli,400.0,2024-11-15,2024-12-11
2,SCORTA,4407DN,4720.0,2024-11-15,2024-12-11
3,MARYGOLD,X23-1299DN mini,900.0,2024-11-15,2024-12-28
4,CALZIFICIO TEX STYLE,4407DN,80.0,2024-11-15,2024-12-11
5,DUELEGS,X23-4493S+ZF,2530.0,2024-11-15,2024-12-11
6,BBF GAMBETTI,2013S+ZN mini,2000.0,2024-11-15,2024-12-11
7,FISHER & PAYKEL,S19-60064V3DN-U427,800.0,2024-11-15,2024-12-11
8,CALZIFICIO GENNY,X24-4403DN,2400.0,2024-11-15,2024-12-11
